In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df_ = pd.read_csv("../input/arl-project/armut_data.csv", encoding='utf-8')

In [3]:
df = df_

In [4]:
def quick_info(dataframe):
    print("--------- HEAD ---------")
    print(dataframe.head(10))
    print("--------- SHAPE ---------")
    print(dataframe.shape)
    print("--------- COLUMNS ---------")
    print(dataframe.columns)
    print("--------- DESCRIBE ---------")
    print(dataframe.describe().T)
    print("--------- INFO ---------")
    print(dataframe.info())
    print("--------- FREQUENCY ---------")
    print(dataframe.nunique())
    print("--------- ANY NULL VALUES ---------")
    print(dataframe.isnull().values.any())
    print("--------- SUM OF NULL VALUES ---------")
    print(dataframe.isnull().sum())

quick_info(df)

--------- HEAD ---------
   UserId  ServiceId  CategoryId           CreateDate
0   25446          4           5  2017-08-06 16:11:00
1   22948         48           5  2017-08-06 16:12:00
2   10618          0           8  2017-08-06 16:13:00
3    7256          9           4  2017-08-06 16:14:00
4   25446         48           5  2017-08-06 16:16:00
5   14354         15           1  2017-08-06 16:27:00
6   14162         21           5  2017-08-06 16:28:00
7   21230         46           4  2017-08-06 16:34:00
8   25446          6           7  2017-08-06 16:39:00
9   10659          4           5  2017-08-06 16:44:00
--------- SHAPE ---------
(162523, 4)
--------- COLUMNS ---------
Index(['UserId', 'ServiceId', 'CategoryId', 'CreateDate'], dtype='object')
--------- DESCRIBE ---------
               count          mean          std  min     25%      50%  \
UserId      162523.0  13089.803862  7325.816060  0.0  6953.0  13139.0   
ServiceId   162523.0     21.641140    13.774405  0.0    13.0     

In [5]:
df["Service"] = df[['ServiceId', 'CategoryId']].apply(lambda x: "_".join(x.astype(str)), axis=1)

In [6]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Service
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [7]:
df['CreateDate'] = pd.to_datetime(df["CreateDate"], errors='coerce') #errors='coerce': If the conversion fails for any particular string then those rows are set to NaT

df['Year'] = df['CreateDate'].dt.year
df['Month'] = df['CreateDate'].dt.month

df["NewDate"] = df[['Year', 'Month']].apply(lambda x: "-".join(x.astype(str)), axis=1)

In [8]:
df.drop(["Year", "Month"], axis=1)

,UserId,ServiceId,CategoryId,CreateDate,Service,NewDate
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-8
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-8
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-8
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-8
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-8
...,...,...,...,...,...,...
162518,10591,25,0,2018-08-06 14:40:00,25_0,2018-8
162519,10591,2,0,2018-08-06 14:43:00,2_0,2018-8
162520,10591,31,6,2018-08-06 14:47:00,31_6,2018-8
162521,12666,38,4,2018-08-06 16:01:00,38_4,2018-8


In [9]:
df["BasketID"] = df[["UserId", "NewDate"]].apply(lambda x: "_".join(x.astype(str)), axis=1)

In [10]:
df = df.groupby(['BasketID', 'Service'])['Service'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [11]:
df

Service,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,...,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
BasketID,,,,,,,,,,,,,,,,,,,,,
0_2017-8,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
0_2017-9,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
0_2018-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0_2018-4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000_2017-12,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99_2017-12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
frequent_items = apriori(df, min_support=0.01, use_colnames=True)

/opt/conda/lib/python3.7/site-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


In [13]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1)

In [14]:
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(25_0),(2_0),0.042895,0.130286,0.013437,0.313257,2.404371,0.007849,1.266432
16,(38_4),(2_0),0.066568,0.130286,0.011191,0.168108,1.290295,0.002518,1.045465
17,(2_0),(38_4),0.130286,0.066568,0.011191,0.085893,1.290295,0.002518,1.021140
18,(38_4),(9_4),0.066568,0.041393,0.010067,0.151234,3.653623,0.007312,1.129413
19,(9_4),(38_4),0.041393,0.066568,0.010067,0.243216,3.653623,0.007312,1.233418


In [15]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]


arl_recommender(rules, '2_0', 2)

['22_0', '25_0']